---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [8]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [9]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [10]:
def graph_identification():
    #algoList = []
    """
    for i in range(0, 5):
        print ("graph ", i+1)
        #print (i)
        G = P1_Graphs[i]
        degrees = G.degree()
        degree_values = sorted(set(degrees.values()))
        histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]
        #algoList.append("SW_L")
        import matplotlib.pyplot as plt
        plt.bar(degree_values,histogram)
        plt.xlabel('Degree')
        plt.ylabel('Fraction of Nodes')
        plt.show()
        
    """
    
    return ["PA", "SW_L", "SW_L", "PA", "SW_H"]

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [11]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [12]:
def salary_predictions():
    import pandas as pd
    import math
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.linear_model import LogisticRegression
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.svm import SVC
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.neural_network import MLPClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.datasets import make_classification, make_blobs
    from sklearn.dummy import DummyClassifier
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    from sklearn.metrics import classification_report
    from sklearn.metrics import roc_curve, auc
    import seaborn as sns
    import matplotlib.pyplot as plt
    from sklearn.metrics import confusion_matrix
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.metrics import roc_auc_score
    from sklearn.model_selection import GridSearchCV
    
    node_labels = nx.get_node_attributes(G,'ManagementSalary')
    
    nodes = G.nodes()
    
    dfList = []
    
    for i in G:
        #print (i, node_labels[i])
        dfList.append([i, node_labels[i]])

    df = pd.DataFrame(dfList)
    df = df.rename(columns={0: 'Node', 1: 'ManagementSalary'})
    
    #centrality measures
    #normalized
    dc = nx.degree_centrality(G)
    close = nx.closeness_centrality(G)
    between = nx.betweenness_centrality(G)
    cluster =  nx.clustering(G)
    
    #map the diciotnaries
    df["DegreeCentrality"] = df["Node"].map(dc)
    df["CloseCentrality"] = df["Node"].map(close)
    df["BetweenCentrality"] = df["Node"].map(between)
    df["Cluster"] = df["Node"].map(cluster)
    
    
    #create the train and test dataset
    df["ManagementSalary"] = df["ManagementSalary"].astype(str).map(str.strip)
    
    dfPred = df[df["ManagementSalary"] == "nan"]
    dfTrain = df[df["ManagementSalary"] != "nan"]

    #print (len(dfPred))
    #print (len(dfTrain))
    #print (len(df))
    
    y_train = dfTrain["ManagementSalary"]
    X_train = dfTrain.drop('ManagementSalary', 1)
    X_train = X_train.drop('Node', 1)
    
    #y_test = dfPred["ManagementSalary"]
    X_test = dfPred.drop('ManagementSalary', 1)
    X_test = X_test.drop('Node', 1)
   
    clf = GradientBoostingClassifier(random_state = 0)
    clf.fit(X_train, y_train)


    #AccuracyAndAUC(model, clf, X_test, y_test, a, b)
    
    
    clf_predicted = clf.predict(X_test)
    result  =   clf.predict_proba(X_test)

    #assign positive class probability predictions
    FinalResult = X_test.copy()
    FinalResult["ProbSalary"] = result[:,1]
    
    #output = FinalResult[["ProbSalary"]]
    
    output = pd.Series(FinalResult['ProbSalary'].values, index=FinalResult.index)


    return output

salary_predictions()

1       0.009446
2       0.987882
5       0.987882
8       0.096666
14      0.054198
18      0.076523
27      0.035641
30      0.862959
31      0.376150
34      0.014332
37      0.028702
40      0.013363
45      0.007258
54      0.493691
55      0.532283
60      0.058701
62      0.987882
65      0.985407
77      0.015035
79      0.018919
97      0.010238
101     0.002733
103     0.618230
108     0.011068
113     0.029088
122     0.002663
141     0.129371
142     0.987882
144     0.010238
145     0.829449
          ...   
913     0.011334
914     0.009482
915     0.000941
918     0.029821
923     0.013088
926     0.035167
931     0.006624
934     0.000930
939     0.000941
944     0.000941
945     0.013088
947     0.027485
950     0.017785
951     0.004950
953     0.002715
959     0.000941
962     0.000941
963     0.024428
968     0.027485
969     0.029941
974     0.023532
984     0.000941
987     0.023532
989     0.027485
991     0.027485
992     0.000941
994     0.000941
996     0.0009

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [13]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})

#print (len(future_connections))
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [50]:
def new_connections_predictions():
    import pandas as pd
    import math
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.linear_model import LogisticRegression
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.svm import SVC
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.neural_network import MLPClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.datasets import make_classification, make_blobs
    from sklearn.dummy import DummyClassifier
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    from sklearn.metrics import classification_report
    from sklearn.metrics import roc_curve, auc
    import seaborn as sns
    import matplotlib.pyplot as plt
    from sklearn.metrics import confusion_matrix
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.metrics import roc_auc_score
    from sklearn.model_selection import GridSearchCV
    
    
    df = future_connections.copy()
    
    df["Future Connection"] = df["Future Connection"].astype(str).map(str.strip)
    
    #create common neighboors didctionary
    common_neigh = [(e[0], e[1], len(list(nx.common_neighbors(G, e[0],e[1])))) for e in nx.non_edges(G)]
    linkDic = {}
    for i in range(0, len(common_neigh)):
        linkDic[(common_neigh[i][0], common_neigh[i][1])] = common_neigh[i][2]
        
    df["Index"] = df.index
    
    df["CN"] = df.Index.map(linkDic)


    
    #print ("cn", len(common_neigh))
    
    #create train and test data
    dfPred = df[df["Future Connection"] == "nan"]
    dfTrain = df[df["Future Connection"] != "nan"]
    
    y_train = dfTrain["Future Connection"]
    X_train = dfTrain.drop('Future Connection', 1)
    X_train = X_train.drop('Index', 1)
    
    X_test = dfPred.drop('Future Connection', 1)
    X_test = X_test.drop('Index', 1)
   
    clf = GradientBoostingClassifier(random_state = 0)
    clf.fit(X_train, y_train)
    
    clf_predicted = clf.predict(X_test)
    result  =   clf.predict_proba(X_test)

    #assign positive class probability predictions
    FinalResult = X_test.copy()
    FinalResult["ProbConn"] = result[:,1]
    
    
    output = pd.Series(FinalResult['ProbConn'].values, index=FinalResult.index)

    
    
    #print (len(df))
    #print (len(dfPred))
    #print (len(dfTrain))
    
    
    return output

new_connections_predictions()

(107, 348)    0.036973
(542, 751)    0.012741
(20, 426)     0.647097
(50, 989)     0.012741
(942, 986)    0.012741
(324, 857)    0.012741
(13, 710)     0.218699
(19, 271)     0.218699
(319, 878)    0.012741
(659, 707)    0.012741
(49, 843)     0.012741
(208, 893)    0.012741
(377, 469)    0.012741
(405, 999)    0.020042
(129, 740)    0.020042
(292, 618)    0.020042
(239, 689)    0.012741
(359, 373)    0.012741
(53, 523)     0.036973
(276, 984)    0.012741
(202, 997)    0.012741
(604, 619)    0.036973
(270, 911)    0.012741
(261, 481)    0.111691
(200, 450)    0.751336
(213, 634)    0.012741
(644, 735)    0.036973
(346, 553)    0.012741
(521, 738)    0.012741
(422, 953)    0.020042
                ...   
(672, 848)    0.012741
(28, 127)     0.947835
(202, 661)    0.012741
(54, 195)     0.999946
(295, 864)    0.012741
(814, 936)    0.012741
(839, 874)    0.012741
(139, 843)    0.012741
(461, 544)    0.012741
(68, 487)     0.012741
(622, 932)    0.012741
(504, 936)    0.020042
(479, 528) 